In [ ]:
import time
import pandas as pd
from datetime import datetime
import firebase_admin
from firebase_admin import credentials, firestore

In [ ]:
cred = credentials.Certificate('../../src/config/smart-agri-bbd45-firebase-adminsdk-jfpri-5e03cc48f1.json')

app = firebase_admin.initialize_app(cred)
db = firestore.client()

In [ ]:
bulk_writer = db.bulk_writer()
batch = db.batch()
collection = db.collection('emission_factors')

In [ ]:
def create_data_item(row):
    return {
        "created_time": time.time(),
        "name": row.get("name", ""),
        "identity_title": row.get("identity_title", ""),
        "description": row.get("description", ""),
        "co2": float(row.get("co2", 0)),
        "co2_unit": row.get("co2_unit", "kg/ha"),
        "ch4": float(row.get("ch4", 0)),
        "ch4_unit": row.get("ch4_unit", "kg/ha"),
        "n2o": float(row.get("n2o", 0)),
        "n2o_unit": row.get("n2o_unit", "kg/ha"),
        "source": row.get("source", None),
        "link": row.get("link", None),
        "type": row.get("type", None),
        "scope": int(row.get("scope", 1)),
    }

In [ ]:
def write_data_to_firestore(df):
    bulk_writer = db.bulk_writer()

    for index, row in df.iterrows():
        print(f"index: {index}")
        time.sleep(0.5)
        data_item = create_data_item(row)
        doc_ref = db.collection("emission_factors").document()
        print(f"data: {data_item}")
        print("________")
        bulk_writer.set(doc_ref, data_item)

    bulk_writer.flush()

In [ ]:
df = pd.read_excel('../data/EF_Table.xlsx')

In [ ]:
def process_name(name):
    words = name.split(" ")
    words = [word.lower() for word in words]
    title = "_".join(words)
    return title

In [ ]:
df["identity_title"] = df["name"].apply(process_name)

In [ ]:
df

In [ ]:
df.shape

In [ ]:
# df = pd.read_excel('../data/EF_Table.xlsx')
start = time.time()
write_data_to_firestore(df)
end = time.time()
print(f'Execution time: {end - start} seconds')

In [ ]:
df.to_excel("EF_Table_v2.xlsx", index=False)

In [ ]:
col_ref = db.collection("emission_factors")

In [ ]:
res = col_ref.stream()

In [ ]:
docs = [{"id": doc.id, **doc.to_dict()} for doc in res]

In [ ]:
docs